In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

earth_density    = 5.51   # [g/cm**3]

In [3]:
dataEA = pd.read_csv(r"C:\Users\silke\OneDrive\Skrivebord\Thesis\Exoplanet_Archive.csv",delimiter=",")
dataLI = pd.read_csv(r"C:\Users\silke\OneDrive\Skrivebord\Thesis\STPM_improved_210721.csv",delimiter=",")
planet_name_LI        = list(dataLI["Star"]+" "+dataLI["Planet"])

earth_like_rocky  = pd.read_csv(r"C:\Users\silke\OneDrive\Skrivebord\Thesis\Mass_Radius_Curves\Earth-like Rocky (32.5% Fe+67.5% MgSiO3).txt",delimiter = "\t")
H20_50_500K       = pd.read_csv(r"C:\Users\silke\OneDrive\Skrivebord\Thesis\Mass_Radius_Curves\50% H2O (50% Earth-like rocky core + 50% H2O layer by mass)_500 K_1mili-bar surface temprture.txt",delimiter = "\t")

In [4]:
# Luque Impoved
planet_id_LI         = dataLI.ID
star_name_LI         = dataLI.Star
planet_letter_LI     = dataLI.Planet

radius_LI            = dataLI.R_Rterra            # [Earth Radius]
radius_unc_up_LI     = dataLI.euR_Rterra          # [Earth Radius] Upper Unc.
radius_unc_low_LI    = dataLI.edR_Rterra          # [Earth Radius] Lower Unc.

mass_LI              = dataLI.M_Mterra            # [Earth Mass] - Planet Mass or Mass*sin(i) [Earth Mass]
mass_unc_up_LI       = dataLI.euM_Mterra          # [Earth Mass] Upper Unc.
mass_unc_low_LI      = dataLI.edM_Mterra          # [Earth Mass] Lower Unc.

density_LI           = dataLI[dataLI.columns[32]]  # [g/cm**3]
density_unc_up_LI    = dataLI[dataLI.columns[33]]  # [g/cm**3] Planet Density Upper Unc. 
density_unc_low_LI   = dataLI[dataLI.columns[34]]  # [g/cm**3] Planet Density Lower Unc
pl_eq_LI             = dataLI.Teq_K                # [K] Equilibrium Temperature


# Exoplanet Archive
planet_name_EA       = dataEA.pl_name
planet_id_EA         = list(range(0,len(planet_name_EA)))

radius_EA            = dataEA.pl_rade            # [Earth Radius]
period_EA            = dataEA.pl_orbper          # [days]
radius_unc_up_EA     = dataEA.pl_radeerr1        # [Earth Radius] Upper Unc.
radius_unc_low_EA    = dataEA.pl_radeerr2        # [Earth Radius] Lower Unc.
mass_EA              = dataEA.pl_bmasse          # [Earth Mass] - Planet Mass or Mass*sin(i) [Earth Mass]

mass_unc_up_EA       = dataEA.pl_bmasseerr1      # [Earth Mass] Upper Unc.
mass_unc_low_EA      = dataEA.pl_bmasseerr2      # [Earth Mass] Lower Unc.

density_EA           = dataEA.pl_dens            # [g/cm**3]
density_unc_up_EA    = dataEA.pl_denserr1        # [g/cm**3] Planet Density Upper Unc. 
density_unc_low_EA   = dataEA.pl_denserr2        # [g/cm**3] Planet Density Lower Unc. 
st_spectype_EA       = dataEA.st_spectype        # Spectral type of star
pl_eq_EA             = dataEA.pl_eqt             # [K] Equilibrium Temperature

for i in range(0,len(planet_name_LI)):
    if planet_name_LI[i] == 'CD-60 8051 b':
        planet_name_LI[i] = 'L 168-9 b'
    if planet_name_LI[i] == 'GJ 3053 b':
        planet_name_LI[i] = 'LHS 1140 b'
    if planet_name_LI[i] == 'GJ 3053 c':
        planet_name_LI[i] = 'LHS 1140 c'

##### Use the Luque values for planets in that sample #### 
for i in range(0,len(planet_name_EA)):
    if planet_name_EA[i] in planet_name_LI:
        LI_index              = planet_name_LI.index(planet_name_EA[i])
        
        pl_eq_EA[i]           = pl_eq_LI[LI_index]
        mass_EA[i]            = mass_LI[LI_index]
        mass_unc_up_EA[i]     = mass_unc_up_LI[LI_index]
        mass_unc_low_EA[i]    = mass_unc_low_LI[LI_index]
        
        radius_EA[i]          = radius_LI[LI_index]
        radius_unc_up_EA[i]   = radius_unc_up_LI[LI_index]      
        radius_unc_low_EA[i]  = radius_unc_low_LI[LI_index]    

        density_EA[i]         = density_LI[LI_index]  
        density_unc_up_EA[i]  = density_unc_up_LI[LI_index]  
        density_unc_low_EA[i] = density_unc_low_LI[LI_index] 
        st_spectype_EA[i]     = "M"

# Filters

### Choose LIV34

In [5]:
def liv34(EA_planets):
    LIV34_planets = ["K2-146 b",'TRAPPIST-1 h', 'TRAPPIST-1 d', 'TRAPPIST-1 e', 'TRAPPIST-1 f', 'TRAPPIST-1 g', 'TRAPPIST-1 c', 'TRAPPIST-1 b', 'GJ 1132 b', 'TOI-270 b', 'LHS 1140 c', 'GJ 1252 b', 'GJ 357 b', 'LTT 3780 b', 'LHS 1478 b', 'GJ 486 b', 'GJ 3473 b', 'L 168-9 b', 'L 98-59 c', 'L 98-59 d', 'LHS 1140 b', 'TOI-270 d', 'TOI-776 b', 'TOI-1634 b', 'TOI-1685 b', 'TOI-1235 b', 'K2-146 c', 'LTT 3780 c', 'TOI-270 c', 'K2-18 b', 'TOI-269 b', 'GJ 1214 b', 'K2-25 b', 'TOI-1231 b'] 
    planet      = []; planet_i      = []; planet_eq   = []; 
    radius      = []; radius_up     = []; radius_low  = []
    mass        = []; mass_up       = []; mass_low    = []
    density     = []; density_up    = []; density_low = []
    
    for i in range(0,len(EA_planets)):
        if EA_planets[i] in LIV34_planets:
            index  = list(planet_name_EA).index(EA_planets[i]); planet.append(dataEA.pl_name[index])
            planet_eq.append(pl_eq_EA[i])
            radius.append(dataEA.pl_rade[index]); radius_up.append(dataEA.pl_radeerr1[index]); radius_low.append(dataEA.pl_radeerr2[index])
            density.append(dataEA.pl_dens[index]); density_up.append(dataEA.pl_denserr1[index]); density_low.append(dataEA.pl_denserr2[index])
            mass.append(dataEA.pl_bmasse[index]); mass_up.append(dataEA.pl_bmasseerr1[index]);  mass_low.append(dataEA.pl_bmasseerr2[index])
    return index,planet,radius,radius_up,radius_low,density,density_up,density_low,mass,mass_up,mass_low,planet_eq

## Plot data

In [6]:
index,planet,radius,radius_up,radius_low,density,density_up,density_low,mass,mass_up,mass_low,planet_eq = liv34(planet_name_EA)
planet2      = []; planet_i2       = []; planet_eq2   = []
radius2      = []; radius_up2      = []; radius_low2  = []
mass2        = []; mass_up2        = []; mass_low2    = []
density2     = []; density_up2     = []; density_low2 = []

radius2 = []
planet2 = []
for i in range(0,len(planet_name_EA)):
    if ((radius_unc_up_EA[i] + radius_unc_up_EA[i])*0.5) / radius_EA[i] < 0.2 and  radius_EA[i]<4 and "M" in str(st_spectype_EA[i]):
        if ((mass_unc_up_EA[i] + mass_unc_up_EA[i])*0.5) / mass_EA[i] < 0.2:
            if planet_name_EA[i] not in planet:
                planet_eq2.append(pl_eq_EA[i]); planet2.append(planet_name_EA[i])
                radius2.append(dataEA.pl_rade[i]); radius_up2.append(dataEA.pl_radeerr1[i]); radius_low2.append(dataEA.pl_radeerr2[i])
                density2.append(dataEA.pl_dens[i]); density_up2.append(dataEA.pl_denserr1[i]); density_low2.append(dataEA.pl_denserr2[i])
                mass2.append(dataEA.pl_bmasse[i]); mass_up2.append(dataEA.pl_bmasseerr1[i]);  mass_low2.append(dataEA.pl_bmasseerr2[i])
                
mass_error_plus = mass_up
mass_error_minus = np.abs(mass_low)
radius_error_plus = radius_up
radius_error_minus = np.abs(radius_low)
density_error_plus = np.divide(density_up,earth_density)
density_error_minus = np.divide(np.abs(density_low),earth_density)
planet_eq = planet_eq 

mass_error_plus2 = mass_up2
mass_error_minus2 = np.abs(mass_low2)
density_error_plus2 = np.divide(density_up2,earth_density)
density_error_minus2= np.divide(np.abs(density_low2),earth_density)
radius_error_plus2 = radius_up2
radius_error_minus2 = np.abs(radius_low2)

planet3      = []; planet_i3       = []; planet_eq3   = []
radius3      = []; radius_up3      = []; radius_low3  = []
mass3        = []; mass_up3        = []; mass_low3    = []
density3     = []; density_up3     = []; density_low3 = []
for i in range(0,len(planet_name_EA)):
    if ((radius_unc_up_EA[i] + radius_unc_up_EA[i])*0.5) / radius_EA[i] < 0.33 and  radius_EA[i]<4 and "M" in str(st_spectype_EA[i]):
        if ((mass_unc_up_EA[i] + mass_unc_up_EA[i])*0.5) / mass_EA[i] < 0.33:
            if planet_name_EA[i] not in planet:
                planet_eq3.append(pl_eq_EA[i]); planet3.append(planet_name_EA[i])
                radius3.append(dataEA.pl_rade[i]); radius_up3.append(dataEA.pl_radeerr1[i]); radius_low3.append(dataEA.pl_radeerr2[i])
                density3.append(dataEA.pl_dens[i]); density_up3.append(dataEA.pl_denserr1[i]); density_low3.append(dataEA.pl_denserr2[i])
                mass3.append(dataEA.pl_bmasse[i]); mass_up3.append(dataEA.pl_bmasseerr1[i]);  mass_low3.append(dataEA.pl_bmasseerr2[i])

mass_error_plus3    = mass_up3
mass_error_minus3   = np.abs(mass_low3)
density_error_plus3 = np.divide(density_up3,earth_density)
density_error_minus3= np.divide(np.abs(density_low3),earth_density)
radius_error_plus3  = radius_up3
radius_error_minus3 = np.abs(radius_low3)

planet_combined     = planet + planet2
planet_eq_combined  = planet_eq + planet_eq2

radius_combined     = radius + radius2
radius_up_combined  = radius_up + radius_up2
radius_low_combined = radius_low + radius_low2

mass_combined      = mass + mass2
mass_up_combined   = mass_up + mass_up2
mass_low_combined  = mass_low + mass_low2

density_combined      = density + density2
density_up_combined   = density_up + density_up2
density_low_combined  = density_low + density_low2

mass_error_plus_combined    = mass_up_combined
mass_error_minus_combined   = np.abs(mass_low_combined)
density_error_plus_combined = np.divide(density_up_combined,earth_density)
density_error_minus_combined= np.divide(np.abs(density_low_combined),earth_density)
radius_error_plus_combined  = radius_up_combined
radius_error_minus_combined = np.abs(radius_low_combined)

planet_combined33     = planet + planet3
planet_eq_combined33  = planet_eq + planet_eq3

radius_combined33     = radius + radius3
radius_up_combined33  = radius_up + radius_up3
radius_low_combined33 = radius_low + radius_low3

mass_combined33      = mass + mass3
mass_up_combined33   = mass_up + mass_up3
mass_low_combined33  = mass_low + mass_low3

density_combined33      = density + density3
density_up_combined33   = density_up + density_up3
density_low_combined33  = density_low + density_low3

mass_error_plus_combined33    = mass_up_combined33
mass_error_minus_combined33   = np.abs(mass_low_combined33)
density_error_plus_combined33 = np.divide(density_up_combined33,earth_density)
density_error_minus_combined33= np.divide(np.abs(density_low_combined33),earth_density)
radius_error_plus_combined33  = radius_up_combined33
radius_error_minus_combined33 = np.abs(radius_low_combined33)


comined_label = []
for i in range(0,len(mass)):
    comined_label.append("LIV34")
for i in range(0,len(mass2)):
    comined_label.append("EA add")

relative_mass_error1 = [error / mass_val if mass_val != 0 else 0 for mass_val, error in zip(mass, mass_error_plus)]
relative_density_error1 = [error / density_val if density_val != 0 else 0 for density_val, error in zip(density, density_error_plus)]
relative_radius_error1 = [error / radius_val if radius_val != 0 else 0 for radius_val, error in zip(radius, radius_error_plus)]